In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd
from implicit.als import AlternatingLeastSquares
import os
from scipy.sparse import csr_matrix


# 1) 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'count'}, inplace=True)

In [8]:
ratings = ratings[['user_id', 'movie_id', 'count']]

In [9]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [10]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [11]:
ratings['movie_id'].nunique()

3628

In [12]:
ratings['user_id'].nunique()

6039

In [13]:
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [14]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Terminator 2: Judgment Day (1991)','Godfather, The (1972)','Forrest Gump (1994)','Matrix, The (1999)','American Beauty (1999)']
favorite_movie_id = movies[movies['title'].isin(my_favorite)]
favorite_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'count':[5,3,7,8,13]})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  
    ratings = ratings.append(favorite_movie)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
352,6041,356,5
585,6041,589,3
847,6041,858,7
2502,6041,2571,8
2789,6041,2858,13


In [15]:
favorite_movie_id

,movie_id,title,genre
352,356,Forrest Gump (1994),Comedy|Romance|War
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
847,858,"Godfather, The (1972)",Action|Crime|Drama
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2789,2858,American Beauty (1999),Comedy|Drama


In [16]:
movie_data = pd.merge(ratings, movies, on='movie_id')
movie_data

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


# CSR Matrix

In [17]:
num_user = ratings['user_id'].nunique()
num_artist = ratings['movie_id'].nunique()

In [18]:
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
352,6041,356,5
585,6041,589,3
847,6041,858,7
2502,6041,2571,8


In [19]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# MF Model Train

In [20]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [21]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [24]:
movie_vector, godfather_vector = als_model.user_factors[6041], als_model.item_factors[858]

In [25]:
movie_vector

array([-0.06541172, -0.10171866, -0.87667954,  0.30963486,  0.30190116,
       -0.226877  , -0.53332925, -0.32713658,  0.3288488 ,  1.1704558 ,
       -0.17008846,  0.07164093, -0.2591294 ,  0.22618143,  0.33053595,
       -0.22289704, -0.14560112,  1.2577465 , -0.91875726, -0.3510067 ,
        0.95382625,  1.1114045 ,  0.2923682 ,  0.49430048,  0.5336761 ,
        0.59735227,  0.8683381 , -0.77992666, -0.61657006,  0.6251981 ,
       -0.20835613, -0.20963067, -0.73386663, -0.10228366,  0.34185934,
       -0.6751514 , -0.01694844,  0.27355218, -0.78609496,  0.9772691 ,
        0.47131464,  0.25144482,  0.60009295,  0.27085423, -0.21985926,
       -0.6255205 ,  0.6613289 ,  1.1695893 ,  0.38684043, -0.68877137,
       -0.3694443 , -0.10494301, -0.53645384, -0.21762794,  0.03476877,
        0.33383077, -0.11581751,  0.19111663,  0.72608554,  0.01004393,
        1.4225763 ,  0.50279   , -0.43352032,  0.57787067, -0.09145988,
        0.46637374, -1.1849452 ,  1.6255608 ,  0.07297716, -0.16

In [26]:
godfather_vector

array([ 0.00996481,  0.02413899, -0.03247602,  0.02849475,  0.01369866,
       -0.02968596,  0.03528881, -0.02609684,  0.00734462,  0.03507073,
        0.04486179,  0.00996869, -0.01575099, -0.01097087,  0.01184283,
       -0.01746006,  0.00110719,  0.04850711, -0.03875883, -0.04412732,
        0.02433516,  0.03643351,  0.00049426,  0.04756562,  0.04031458,
       -0.00103579,  0.0087517 , -0.02311929, -0.03245016,  0.00619121,
        0.00950624, -0.00636259, -0.02174195, -0.00306722, -0.00197916,
        0.00370738,  0.01440545,  0.03937354, -0.02866037,  0.0203537 ,
        0.03872946,  0.00453694,  0.05055008,  0.02626996,  0.02207542,
       -0.05266163,  0.02492419, -0.01666913,  0.04454458,  0.00678352,
       -0.03044653, -0.00780539, -0.04041018,  0.01896588,  0.00786606,
        0.03773379, -0.01496411,  0.01880542,  0.01076723, -0.00980057,
       -0.00276409,  0.03503954, -0.00523462,  0.03927006,  0.01247647,
        0.03191671, -0.00775818,  0.03862762,  0.01654926,  0.02

In [27]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(movie_vector, godfather_vector)

0.7122679

In [28]:
jumanji_vector = als_model.item_factors[2]
np.dot(movie_vector, jumanji_vector)

-0.03847047

# 비슷한 아티스트 찾기 + 추천

In [29]:
favorite_movie = 'American Beauty (1999)'
movie_id = movies[movies['title']=='American Beauty (1999)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(2858, 1.0000001),
 (2997, 0.7400964),
 (2599, 0.46555766),
 (2396, 0.45649838),
 (608, 0.4042395),
 (1787, 0.3807555),
 (593, 0.37547955),
 (110, 0.3709119),
 (296, 0.36327955),
 (3160, 0.3499165),
 (989, 0.3463014),
 (318, 0.33910415),
 (3890, 0.33712074),
 (2028, 0.3362827),
 (3481, 0.3330451)]

In [31]:
user = 6041
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1221, 0.6076337),
 (1210, 0.5632447),
 (480, 0.46505383),
 (2762, 0.45110446),
 (110, 0.43221217),
 (260, 0.42536655),
 (1580, 0.4141246),
 (1196, 0.4141151),
 (2028, 0.41303736),
 (2997, 0.39782414),
 (593, 0.35830104),
 (2023, 0.34070298),
 (2628, 0.30491093),
 (457, 0.30014658),
 (1265, 0.27901435),
 (527, 0.27468264),
 (2916, 0.27341777),
 (608, 0.2724977),
 (2396, 0.27097318),
 (1270, 0.2654093)]

In [32]:
movies[movies['movie_id'].isin([i[0] for i in movie_recommended])]

,movie_id,title,genre
108,110,Braveheart (1995),Action|Drama|War
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
453,457,"Fugitive, The (1993)",Action|Thriller
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
523,527,Schindler's List (1993),Drama|War
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller
604,608,Fargo (1996),Crime|Drama|Thriller
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1203,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
